In [115]:
import numpy as np
import pandas as pd

In [99]:
df = pd.read_csv("../data/preprocessed_data.csv")

## Developing a model


In [100]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300259 entries, 0 to 300258
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   airline         300259 non-null  object 
 1   from            300259 non-null  object 
 2   to              300259 non-null  object 
 3   price           300259 non-null  int64  
 4   class           300259 non-null  object 
 5   flight_code     300259 non-null  object 
 6   stops           300259 non-null  object 
 7   departure_time  300259 non-null  object 
 8   arrival_time    300259 non-null  object 
 9   datetime        300259 non-null  object 
 10  dow             300259 non-null  object 
 11  holiday         300259 non-null  int64  
 12  days_until      300259 non-null  int64  
 13  duration        300255 non-null  float64
dtypes: float64(1), int64(3), object(10)
memory usage: 32.1+ MB


In [102]:
# We have null values in the duration column, we will drop them
df = df.dropna()

# We also will drop the datetime column
df = df.drop(columns=["datetime"])

# Let's remove flight code as well
df = df.drop(columns=["flight_code"])

In [103]:
df.sample(3)

,airline,from,to,price,class,stops,departure_time,arrival_time,dow,holiday,days_until,duration
63105,Indigo,Mumbai,Kolkata,9723,economy,1,afternoon,evening,Monday,0,11,395.0
34530,Air India,Delhi,Hyderabad,4209,economy,1,evening,morning,Saturday,0,44,735.0
14027,AirAsia,Delhi,Bangalore,3090,economy,1,morning,evening,Friday,0,22,595.0


In [104]:
categorical_columns = [
    "airline",
    "from",
    "to",
    "class",
#    "flight_code",
    "departure_time",
    "arrival_time",
    "dow",
    "holiday",
]
numerical_columns = ["duration",
                     "days_until"]
target_column = "price"

## Splitting the data into training, validation and testing sets


In [105]:
df_train_val, df_test = train_test_split(df, test_size=0.2, random_state=57)
df_train, df_val = train_test_split(
    df_train_val, test_size=0.25, random_state=57)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [106]:
X_train = df_train[categorical_columns + numerical_columns]
X_val = df_val[categorical_columns +numerical_columns]
X_test = df_test[categorical_columns + numerical_columns]

y_train = df_train[target_column].values
y_val = df_val[target_column].values
y_test = df_test[target_column].values


### Scaling the data

In [108]:
def treat_categorical_cols(df_train: pd.DataFrame, cols: list, dv: DictVectorizer):
    df = df_train.copy()
    df[cols] = df[cols].astype(str)
    train_dict = df[cols].to_dict(orient="records")


    if dv is None:
        dv = DictVectorizer(sparse=False)
        dv.fit(train_dict)
        
    df_cat = dv.transform(train_dict)
    df_train_cat = pd.DataFrame(df_cat, columns=dv.get_feature_names_out())

    df_train_continuous = df_train.drop(columns=cols)
    df_train = pd.concat([df_train_continuous, df_train_cat], axis=1)

    return df_train, dv

In [109]:
X_train, dv = treat_categorical_cols(X_train,
                                     categorical_columns,
                                     dv=None)
X_val = treat_categorical_cols(X_val,
                               categorical_columns,
                               dv=dv)[0]
X_test = treat_categorical_cols(X_test,
                                categorical_columns,
                                dv=dv)[0]


In [111]:
def treat_numerical_columns(df_train: pd.DataFrame, cols: list, scaler: StandardScaler):
    df = df_train.copy()
    if scaler is None:
        scaler = StandardScaler()
        scaler.fit(df[cols])
    df[cols] = scaler.transform(df[cols])
    return df, scaler

In [112]:
X_train, scaler = treat_numerical_columns(X_train,
                                            numerical_columns,
                                            scaler=None)
X_val = treat_numerical_columns(X_val,
                                numerical_columns,
                                scaler=scaler)[0]
X_test = treat_numerical_columns(X_test,
                                 numerical_columns,
                                 scaler=scaler)[0]

## Training the model


In [116]:
LR = LinearRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_val)
print("Linear Regression")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}")
print("R2: ", r2_score(y_val, y_pred))
print()

RF = RandomForestRegressor(n_estimators=25, random_state=57)
RF.fit(X_train, y_train)
y_pred = RF.predict(X_val)
print("Random Forest")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}")
print("R2: ", r2_score(y_val, y_pred))
print()

GB = GradientBoostingRegressor(random_state=57)
GB.fit(X_train, y_train)
y_pred = GB.predict(X_val)
print("Gradient Boosting")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val, y_pred))}")
print("R2: ", r2_score(y_val, y_pred))

Linear Regression
RMSE: 7071.620080374951
R2:  0.9032076917440507
Random Forest
RMSE: 2873.331481671299
R2:  0.9840200848643698
Gradient Boosting
RMSE: 5022.104663293175
R2:  0.951182613533328
